In [8]:
%%capture
pip install praw

In [9]:
%%capture
pip install azure-ai-textanalytics

In [10]:
from collections import Counter

In [11]:
%%capture
import praw
import json

# Competitors & Businesses Data | Alpena, MI | Yelp Search API

In [13]:
import requests
import pandas as pd

def find_businesses(api_key, query, find_loc):
    endpoint = "https://serpapi.com/search"
    params = {
        "api_key": api_key,
        "engine": "yelp",
        "find_desc": query,
        "find_loc": find_loc
    }

    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, Details: {response.text}"

def response_to_dataframe(response_data, county_name):
    if "organic_results" in response_data:
        results_list = [
            {
                'County': county_name,  # Include county name in each row
                'Position': result.get('position', 'N/A'),
                'Place ID': result.get('place_ids', ['N/A'])[0],
                'Title': result.get('title', 'N/A'),
                'Link': result.get('link', 'N/A'),
                'Rating': result.get('rating', 'N/A'),
                'Reviews Count': result.get('reviews', 'N/A'),
                'Categories': ", ".join([category['title'] for category in result.get('categories', [])]),
                'Price': result.get('price', 'N/A'),
                'Phone': result.get('phone', 'N/A'),
                'Snippet': result.get('snippet', 'N/A'),
                'Thumbnail': result.get('thumbnail', 'N/A')
            }
            for result in response_data['organic_results']
        ]
        return pd.DataFrame(results_list)
    else:
        return pd.DataFrame()

api_key = "YOUR_API_KEY"  # Replace with your actual SerpApi API key

# List of counties in Michigan (replace with full list)
michigan_counties = [
    "Alcona County", "Alger County", "Allegan County"  # , ... continue with the rest of the counties
]

query = "supermarket"

# Initialize an empty DataFrame to store results
all_businesses_df = pd.DataFrame()

# Loop through each county and append results to the DataFrame
for county in michigan_counties:
    find_loc = f"{county}, MI"
    response_data = find_businesses(api_key, query, find_loc)
    if isinstance(response_data, dict):  # Check if the response is valid (a dictionary)
        businesses_df = response_to_dataframe(response_data, county)  # Pass county name here
        all_businesses_df = all_businesses_df.append(businesses_df, ignore_index=True)
    else:
        print(response_data)  # Print the error message if not a valid dictionary

# Display the DataFrame
all_businesses_df.head(10)  # Displaying the first few rows for demonstration

,Position,Place ID,Title,Link,Rating,Reviews Count,Categories,Price,Phone,Snippet,Thumbnail
0,1,KSd43YB6H3G8SBi_4EArlQ,Court Yard Ristorante & Olde Roost Lounge,https://www.yelp.com/biz/court-yard-ristorante...,4.5,130,"American, Bars",$$,(989) 356-9511,"We ordered two different entrees, chicken tetr...",https://s3-media0.fl.yelpcdn.com/bphoto/2xLa6Y...
1,2,DX4uoTWPw5qjjOIUMIi64Q,Austin Brothers Beer Co,https://www.yelp.com/biz/austin-brothers-beer-...,4.5,213,"Breweries, New American, Pubs",$$,(989) 340-2300,"One of my favorite places to eat, great food a...",https://s3-media0.fl.yelpcdn.com/bphoto/gvJ8qL...
2,3,pdCZXd-GNYu_qFXdrvGPVA,Red Brick Tap & Barrel,https://www.yelp.com/biz/red-brick-tap-and-bar...,4.7,51,"American, Beer Bar",$$,(989) 340-2175,Absolutely love this restaurant. The food is a...,https://s3-media0.fl.yelpcdn.com/bphoto/2W5-xu...
3,4,cHpl-QJ-t7SQ4ci4YIPCvA,Cabin Creek Coffee,https://www.yelp.com/biz/cabin-creek-coffee-al...,4.3,59,"Coffee & Tea, Breakfast & Brunch, Sandwiches",$,(989) 356-5613,Great cold brew and snacks. I wish they offere...,https://s3-media0.fl.yelpcdn.com/bphoto/Fj8Z2p...
4,5,mdfX2S0uF_EpHUr-s8NZtA,HopSide Brewery,https://www.yelp.com/biz/hopside-brewery-alpena,4.0,22,"Breweries, Pizza, Burgers",N/A,(989) 340-1280,This was a last minute lunch place for a last ...,https://s3-media0.fl.yelpcdn.com/bphoto/WHthns...
5,6,rr5mMvGEI-LH46BnB7MNgw,The Fresh Palate,https://www.yelp.com/biz/the-fresh-palate-alpena,3.7,132,"Bars, Burgers, Tapas/Small Plates",$$,(989) 358-1400,The food was excellent. A lot of healthy optio...,https://s3-media0.fl.yelpcdn.com/bphoto/uvaYhI...
6,7,843oms-I6btHeNFUQrLrKg,John Boy’s Restaurant,https://www.yelp.com/biz/john-boys-restaurant-...,3.8,52,"Breakfast & Brunch, American, Coffee & Tea",$,(989) 356-0481,Cute little restaurant. Maybe 10 tables. Our w...,https://s3-media0.fl.yelpcdn.com/bphoto/WpYBkQ...
7,8,bkSjucvLAGZnNrGYri2BCA,Mangos Tequila Bar,https://www.yelp.com/biz/mangos-tequila-bar-al...,4.3,39,"Cocktail Bars, Mexican",N/A,(989) 340-1232,N/A,https://s3-media0.fl.yelpcdn.com/bphoto/-tMWge...
8,9,YOJWWpvGWE8goUtU5sTK6g,Hungry Hippie,https://www.yelp.com/biz/hungry-hippie-alpena,4.7,19,"Juice Bars & Smoothies, Tacos, Sandwiches",$,(989) 340-1482,Always nice to see a unique new restaurant pop...,https://s3-media0.fl.yelpcdn.com/bphoto/TaTDza...
9,10,u50dtaZ8lGYxkZ_h6j54hg,Pompeyos,https://www.yelp.com/biz/pompeyos-alpena,4.2,56,Mexican,$$,(989) 340-2036,This place was amazing! The food was delicious...,https://s3-media0.fl.yelpcdn.com/bphoto/TK7CMH...


# Reviews Data for Meijer | Alpena, MI | Yelp Reviews API

In [8]:
import requests

def get_yelp_reviews(api_key, place_id):
    endpoint = "https://serpapi.com/search"
    params = {
        "api_key": api_key,
        "engine": "yelp_reviews",
        "place_id": place_id  # Yelp ID of the store
    }

    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, Details: {response.text}"

# Replace 'Your_API_Key' with your actual SerpApi API key
api_key = "acfe31e86da8e66b60e1a7b6ee5ea486fc14f8fe7da7663f43213e81f1aafd0d"
reviews_data = get_yelp_reviews(api_key, "meijer-alpena")
print(reviews_data)

{'search_metadata': {'id': '65ac2fb0defa133c5ff5db42', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/9a7d0c6ed9c53538/65ac2fb0defa133c5ff5db42.json', 'created_at': '2024-01-20 20:40:16 UTC', 'processed_at': '2024-01-20 20:40:16 UTC', 'yelp_reviews_url': 'https://www.yelp.com/biz/meijer-alpena/review_feed?', 'raw_html_file': 'https://serpapi.com/searches/9a7d0c6ed9c53538/65ac2fb0defa133c5ff5db42.html', 'prettify_html_file': 'https://serpapi.com/searches/9a7d0c6ed9c53538/65ac2fb0defa133c5ff5db42.prettify', 'total_time_taken': 1.63}, 'search_parameters': {'engine': 'yelp_reviews', 'place_id': 'meijer-alpena'}, 'search_information': {'total_results': 9, 'language': 'en', 'business': 'Meijer'}, 'review_languages': [{'hl': 'en', 'count': 9}], 'reviews': [{'user': {'name': 'Mic B.', 'user_id': 'rKIOFkkY1hAQhCNYimqg1A', 'link': 'https://www.yelp.com/user_details?userid=rKIOFkkY1hAQhCNYimqg1A', 'thumbnail': 'https://s3-media0.fl.yelpcdn.com/photo/9C45-exy5ZCxv2Y4dZW-mA/60s

In [17]:
import pandas as pd

# Function to transform the reviews data into a DataFrame
def reviews_to_dataframe(reviews_data):
    reviews_list = []

    for review in reviews_data['reviews']:
        review_info = {
            'User Name': review['user']['name'],
            'User ID': review['user']['user_id'],
            'User Link': review['user'].get('link', 'N/A'),
            'User Thumbnail': review['user'].get('thumbnail', 'N/A'),
            'User Address': review['user'].get('address', 'N/A'),
            'User Photos Count': review['user'].get('photos', 'N/A'),
            'User Reviews Count': review['user'].get('reviews', 'N/A'),
            'Review Text': review['comment']['text'],
            'Review Language': review['comment'].get('language', 'N/A'),
            'Review Date': review['date'],
            'Rating': review['rating'],
            'Feedback Useful': review.get('feedback', {}).get('useful', 'N/A'),
            'Feedback Funny': review.get('feedback', {}).get('funny', 'N/A'),
            'Feedback Cool': review.get('feedback', {}).get('cool', 'N/A')
        }
        reviews_list.append(review_info)

    return pd.DataFrame(reviews_list)

# Create DataFrame from the reviews data
reviews_df = reviews_to_dataframe(reviews_data)

# Display the DataFrame
reviews_df.head(10)

,User Name,User ID,User Link,User Thumbnail,User Address,User Photos Count,User Reviews Count,Review Text,Review Language,Review Date,Rating,Feedback Useful,Feedback Funny,Feedback Cool
0,Mic B.,rKIOFkkY1hAQhCNYimqg1A,https://www.yelp.com/user_details?userid=rKIOF...,https://s3-media0.fl.yelpcdn.com/photo/9C45-ex...,"Au Sable, MI",23,41,Love this company. Moved back up north from Fl...,en,1/20/2022,5,N/A,N/A,N/A
1,Mina S.,cff0cU7TniLvipWsYZt8fA,https://www.yelp.com/user_details?userid=cff0c...,https://s3-media0.fl.yelpcdn.com/photo/1mZncdu...,"Saginaw, MI",610,1147,This is such a nice Meijer! It is very clean w...,en,3/10/2018,5,2,N/A,N/A
2,Michael S.,TdRoegolE5NV5hYJrILShQ,https://www.yelp.com/user_details?userid=TdRoe...,https://s3-media0.fl.yelpcdn.com/photo/5Ra5uz9...,"Saginaw, MI",4,99,"Brand new location, naturally everything is fr...",en,5/10/2015,5,3,1,1
3,Arnie S.,wxSHxdFbwLXqPUq9Rr-O4w,https://www.yelp.com/user_details?userid=wxSHx...,https://s3-media0.fl.yelpcdn.com/photo/8sQpmtG...,"Alpena, MI",N/A,2,"This review is for Meijer Optical. Employee, C...",en,3/21/2018,5,N/A,N/A,N/A
4,Pete P.,PxCTlTfaABn78JEuGT7S8g,https://www.yelp.com/user_details?userid=PxCTl...,https://s3-media0.fl.yelpcdn.com/photo/HqdH3qS...,"Detroit, MI",22,355,This has got to be THE friendliest Meijer I ha...,en,8/26/2015,5,3,N/A,N/A
5,Emily G.,TkH7MZLEtqqJ-bQMtHOuZQ,https://www.yelp.com/user_details?userid=TkH7M...,https://s3-media0.fl.yelpcdn.com/photo/DUzyDJ8...,"Port Angeles, WA",10,17,"This is a great store, super clean and bright ...",en,4/19/2016,5,1,N/A,1
6,Sara S.,LwMXGLXYenpw8n-GnRC1DQ,https://www.yelp.com/user_details?userid=LwMXG...,https://s3-media0.fl.yelpcdn.com/photo/9kkXNbq...,"Alpena, MI",5,21,Employees are very friendly. I only hate shopp...,en,12/22/2015,4,N/A,N/A,N/A
7,M H.,LGlk-Zb69sGQwJVu2w4U_Q,https://www.yelp.com/user_details?userid=LGlk-...,https://s3-media0.fl.yelpcdn.com/assets/srv0/y...,"Tampa Bay, FL",253,501,Sooooo happy Meijer is in Alpena as I hate giv...,en,8/24/2016,5,2,N/A,2
8,Katie Y.,qgKScw2dYvvwJkvljXgFGA,https://www.yelp.com/user_details?userid=qgKSc...,https://s3-media0.fl.yelpcdn.com/assets/srv0/y...,"Alpena, MI",N/A,2,Meijer is the place I go to get everything! Th...,en,7/22/2017,5,N/A,N/A,N/A


In [18]:
reviews_list = reviews_df['Review Text'].tolist()

In [19]:
def read_scores(sent_file):
    with open(sent_file) as f:
        return {line.split('\t')[0]: int(line.split('\t')[1]) for line in f}
Scores_Table = read_scores('AFINN-111.txt')

In [20]:
# %%capture
Scores_Table

{'abandon': -2,
 'abandoned': -2,
 'abandons': -2,
 'abducted': -2,
 'abduction': -2,
 'abductions': -2,
 'abhor': -3,
 'abhorred': -3,
 'abhorrent': -3,
 'abhors': -3,
 'abilities': 2,
 'ability': 2,
 'aboard': 1,
 'absentee': -1,
 'absentees': -1,
 'absolve': 2,
 'absolved': 2,
 'absolves': 2,
 'absolving': 2,
 'absorbed': 1,
 'abuse': -3,
 'abused': -3,
 'abuses': -3,
 'abusive': -3,
 'accept': 1,
 'accepted': 1,
 'accepting': 1,
 'accepts': 1,
 'accident': -2,
 'accidental': -2,
 'accidentally': -2,
 'accidents': -2,
 'accomplish': 2,
 'accomplished': 2,
 'accomplishes': 2,
 'accusation': -2,
 'accusations': -2,
 'accuse': -2,
 'accused': -2,
 'accuses': -2,
 'accusing': -2,
 'ache': -2,
 'achievable': 1,
 'aching': -2,
 'acquit': 2,
 'acquits': 2,
 'acquitted': 2,
 'acquitting': 2,
 'acrimonious': -3,
 'active': 1,
 'adequate': 1,
 'admire': 3,
 'admired': 3,
 'admires': 3,
 'admiring': 3,
 'admit': -1,
 'admits': -1,
 'admitted': -1,
 'admonish': -2,
 'admonished': -2,
 'adopt': 

In [26]:
scores = []
for txt in reviews_list:
    count = 0;
    for score in Scores_Table:
         string = json.dumps(txt).lower()
         word_score = string.count(score) * Scores_Table[score]
         count = count + word_score
    scores.append(count)

In [27]:
scores

[10, 23, 26, 1, 27, 36, -1, 19, 17]

In [15]:
reviews_list

NameError: name 'reviews_list' is not defined

In [28]:
import time
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Your Azure Text Analytics API credentials
key = 'a308c79848c54a2cb03e3023ffedff10'
endpoint = 'https://web-data-chinmay.cognitiveservices.azure.com/'

#key = '0e6c8eb16ba24e90bbc7ae50ff912464'
#endpoint = 'https://conordarkknight.cognitiveservices.azure.com/'

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=ta_credential)
    return text_analytics_client

def analyze_sentiments(descriptions):
    client = authenticate_client()
    sentiments = []

    requests_per_second_limit = 100
    requests_per_minute_limit = 300
    txt_number = 0
    time_counter = time.time()

    for txt in descriptions:
        txt_number += 1
        print(f"Processing review {txt_number}...")

        # Detect Sentiments
        response = client.analyze_sentiment(documents=[txt])[0]
        if not response.is_error:
            sentiments.append(response.sentiment)
        else:
            print(f"Error processing review {txt_number}: {response.id}, {response.error}")

        end_time = time.time()

        requests_in_this_second = txt_number % requests_per_second_limit
        requests_in_this_minute = txt_number % requests_per_minute_limit

        # If we reach the limit of requests per second, add a delay to maintain the limit
        if requests_in_this_second == 0:
            time_taken_for_batch = end_time - time_counter
            if time_taken_for_batch < 1:
                time.sleep(1 - time_taken_for_batch)
            time_counter = time.time()

        if requests_in_this_minute == 0:
            time_taken_for_minute = end_time - time_counter
            if time_taken_for_minute < 60:
                time.sleep(60 - time_taken_for_minute)
            time_counter = time.time()

    return sentiments

# Example usage:
sentiments = analyze_sentiments(reviews_list)
print("Sentiments:", sentiments)

Processing review 1...
Processing review 2...
Processing review 3...
Processing review 4...
Processing review 5...
Processing review 6...
Processing review 7...
Processing review 8...
Processing review 9...
Sentiments: ['mixed', 'mixed', 'positive', 'mixed', 'positive', 'positive', 'mixed', 'mixed', 'positive']


In [29]:
value_counts = Counter(sentiments)

# Iterate through the Counter object and print in the desired format
for value, count in value_counts.items():
    print(f"{value} : {count}")

mixed : 5
positive : 4


# Reviews Data for Meijer | Alpena, MI | Google Reviews API

In [31]:
import requests
import json
import time

# Replace with your actual API key
API_KEY = 'acfe31e86da8e66b60e1a7b6ee5ea486fc14f8fe7da7663f43213e81f1aafd0d'

# Replace with the actual place_id of the Meijer store
PLACE_ID = 'ChIJTaHOi-teM00RkiGM0lBe48s'

# Google Maps Reviews API endpoint
base_url = "https://serpapi.com/search.json"

def get_reviews(api_key, place_id, next_page_token=None):
    params = {
        'engine': 'google_maps_reviews',
        'api_key': api_key,
        'place_id': place_id,
        'next_page_token': next_page_token
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to retrieve data:", response.status_code)
        return None

# List to hold all reviews
all_reviews = []

# Initial call without a next_page_token
response_data = get_reviews(API_KEY, PLACE_ID)

while response_data:
    all_reviews.extend(response_data.get('reviews', []))
    next_page_token = response_data.get('serpapi_pagination', {}).get('next_page_token')

    if not next_page_token:
        break

    # To avoid sending too many requests in a short time
    time.sleep(2)

    # Fetch the next page of reviews
    response_data = get_reviews(API_KEY, PLACE_ID, next_page_token)

# Save the data to a JSON file
with open('meijer_reviews.json', 'w') as file:
    json.dump(all_reviews, file, indent=4)

print(f"Total reviews retrieved: {len(all_reviews)}")
print("Reviews data saved to 'meijer_reviews.json'")

Failed to retrieve data: 429
Total reviews retrieved: 940
Reviews data saved to 'meijer_reviews.json'


In [33]:
import pandas as pd

# Load the JSON data from the file
file_path = 'meijer_reviews.json'
with open(file_path, 'r') as file:
    reviews_data = json.load(file)

# Create a DataFrame
df = pd.DataFrame(reviews_data)

# Display the DataFrame
pd.set_option('display.max_columns', None)
df.head(10)

,link,rating,date,images,source,review_id,user,snippet,likes,details,response
0,https://www.google.com/maps/reviews/data=!4m8!...,1.0,a month ago,[https://lh5.googleusercontent.com/p/AF1QipNEK...,Google,ChdDSUhNMG9nS0VJQ0FnSURsa1B5THZBRRAB,"{'name': 'Jena Bennena', 'link': 'https://www....",Bought garlic aioli the other night and it was...,NaN,NaN,NaN
1,https://www.google.com/maps/reviews/data=!4m8!...,3.0,2 days ago,NaN,Google,ChZDSUhNMG9nS0VJQ0FnSUROa0xmbUNREAE,"{'name': 'Anna Ratz Nelson', 'link': 'https://...",First...just not a fan of big stores like this...,NaN,NaN,NaN
2,https://www.google.com/maps/reviews/data=!4m8!...,5.0,a week ago,NaN,Google,ChZDSUhNMG9nS0VJQ0FnSUNONmNTLUhnEAE,"{'name': 'Barb Morris', 'link': 'https://www.g...",I love their variety and the different style a...,NaN,NaN,NaN
3,https://www.google.com/maps/reviews/data=!4m8!...,5.0,3 months ago,NaN,Google,ChZDSUhNMG9nS0VJQ0FnSUQ1N1B2bGFBEAE,"{'name': 'Judi Brown', 'link': 'https://www.go...",I love Meijer. They carry everything I need. ...,NaN,NaN,NaN
4,https://www.google.com/maps/reviews/data=!4m8!...,5.0,4 months ago,NaN,Google,ChZDSUhNMG9nS0VJQ0FnSUNLb09LN0hnEAE,"{'name': 'Cory Bennett', 'link': 'https://www....",My recent trip to Meijer was a delightful shop...,NaN,NaN,NaN
5,https://www.google.com/maps/reviews/data=!4m8!...,5.0,a year ago,[https://lh5.googleusercontent.com/p/AF1QipMbH...,Google,ChZDSUhNMG9nS0VJQ0FnSUR1XzdhQVV3EAE,"{'name': 'Eely Dal', 'link': 'https://www.goog...","For me, this place is a gem. Their produce dep...",NaN,NaN,NaN
6,https://www.google.com/maps/reviews/data=!4m8!...,5.0,2 months ago,NaN,Google,ChZDSUhNMG9nS0VJQ0FnSURGd19IUkJnEAE,"{'name': 'Christine Quade', 'link': 'https://w...","I had left a bag of groceries a week ago, they...",NaN,NaN,NaN
7,https://www.google.com/maps/reviews/data=!4m8!...,5.0,3 months ago,NaN,Google,ChdDSUhNMG9nS0VJQ0FnSUM1cEtyWnlRRRAB,"{'name': 'Salem Campbell', 'link': 'https://ww...",I always find what I need here. Except that th...,NaN,NaN,NaN
8,https://www.google.com/maps/reviews/data=!4m8!...,5.0,a month ago,NaN,Google,ChZDSUhNMG9nS0VJQ0FnSUMxMklldFB3EAE,"{'name': 'Nicholas Tyrance', 'link': 'https://...","Great shopping experience, tons of items I hav...",NaN,NaN,NaN
9,https://www.google.com/maps/reviews/data=!4m8!...,5.0,2 months ago,NaN,Google,ChZDSUhNMG9nS0VJQ0FnSURGMzZMUllBEAE,"{'name': 'Djuan', 'link': 'https://www.google....","Trevor, the night stocker. I asked him if he w...",NaN,NaN,NaN


In [34]:
greviews_list = df['snippet'].tolist()

In [35]:
greviews_list

['Bought garlic aioli the other night and it was so spoiled that it had turned brown and runny. I thought at first from looking at the bottom it had just not been shaken enough but after removing the seal I had never been more disappointed and disgusted.',
 'First...just not a fan of big stores like this.  But, it was clean and well stocked.  Had personnel out on floor for any questions I had.  Prices seem a bit high.  Not impressed with the layout of the grocery side.  Seems hit and miss.  They must be doing something right though...it was busy.',
 'I love their variety and the different style and quality of clothing they offer, along with their clearance racks. Also, their variety of food, personal hygiene products and pretty much whatever you need.',
 "I love Meijer.  They carry everything I need.  Although I went through the self check out and every register was being used the associate seen I had two beverage boxes she said I didn't have to lift them and she went and got the scann

In [36]:
def read_scores(sent_file):
    with open(sent_file) as f:
        return {line.split('\t')[0]: int(line.split('\t')[1]) for line in f}
Scores_Table = read_scores('AFINN-111.txt')

In [37]:
scores = []
for txt in greviews_list:
    count = 0;
    for score in Scores_Table:
         string = json.dumps(txt).lower()
         word_score = string.count(score) * Scores_Table[score]
         count = count + word_score
    scores.append(count)

In [38]:
import time
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Your Azure Text Analytics API credentials
key = 'a308c79848c54a2cb03e3023ffedff10'
endpoint = 'https://web-data-chinmay.cognitiveservices.azure.com/'

#key = '0e6c8eb16ba24e90bbc7ae50ff912464'
#endpoint = 'https://conordarkknight.cognitiveservices.azure.com/'

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=ta_credential)
    return text_analytics_client

def analyze_sentiments(descriptions):
    client = authenticate_client()
    sentiments = []

    requests_per_second_limit = 100
    requests_per_minute_limit = 300
    txt_number = 0
    time_counter = time.time()

    for txt in descriptions:
        txt_number += 1
        print(f"Processing review {txt_number}...")

        # Detect Sentiments
        response = client.analyze_sentiment(documents=[txt])[0]
        if not response.is_error:
            sentiments.append(response.sentiment)
        else:
            print(f"Error processing review {txt_number}: {response.id}, {response.error}")

        end_time = time.time()

        requests_in_this_second = txt_number % requests_per_second_limit
        requests_in_this_minute = txt_number % requests_per_minute_limit

        # If we reach the limit of requests per second, add a delay to maintain the limit
        if requests_in_this_second == 0:
            time_taken_for_batch = end_time - time_counter
            if time_taken_for_batch < 1:
                time.sleep(1 - time_taken_for_batch)
            time_counter = time.time()

        if requests_in_this_minute == 0:
            time_taken_for_minute = end_time - time_counter
            if time_taken_for_minute < 60:
                time.sleep(60 - time_taken_for_minute)
            time_counter = time.time()

    return sentiments

# Example usage:
sentiments = analyze_sentiments(greviews_list)
print("Sentiments:", sentiments)

Processing review 1...
Processing review 2...
Processing review 3...
Processing review 4...
Processing review 5...
Processing review 6...
Processing review 7...
Processing review 8...
Processing review 9...
Processing review 10...
Processing review 11...
Processing review 12...
Processing review 13...
Processing review 14...
Processing review 15...
Processing review 16...
Processing review 17...
Processing review 18...
Processing review 19...
Processing review 20...
Processing review 21...
Processing review 22...
Processing review 23...
Processing review 24...
Processing review 25...
Processing review 26...
Processing review 27...
Processing review 28...
Processing review 29...
Processing review 30...
Processing review 31...
Processing review 32...
Processing review 33...
Processing review 34...
Processing review 35...
Processing review 36...
Processing review 37...
Processing review 38...
Processing review 39...
Processing review 40...
Processing review 41...
Processing review 42...
P

Processing review 335...
Processing review 336...
Processing review 337...
Processing review 338...
Processing review 339...
Processing review 340...
Processing review 341...
Processing review 342...
Processing review 343...
Processing review 344...
Processing review 345...
Processing review 346...
Processing review 347...
Processing review 348...
Processing review 349...
Processing review 350...
Processing review 351...
Processing review 352...
Processing review 353...
Processing review 354...
Processing review 355...
Processing review 356...
Processing review 357...
Processing review 358...
Processing review 359...
Processing review 360...
Processing review 361...
Processing review 362...
Processing review 363...
Processing review 364...
Processing review 365...
Processing review 366...
Processing review 367...
Processing review 368...
Processing review 369...
Processing review 370...
Processing review 371...
Processing review 372...
Processing review 373...
Processing review 374...


Processing review 664...
Processing review 665...
Processing review 666...
Processing review 667...
Processing review 668...
Processing review 669...
Processing review 670...
Processing review 671...
Processing review 672...
Processing review 673...
Processing review 674...
Processing review 675...
Processing review 676...
Processing review 677...
Processing review 678...
Processing review 679...
Processing review 680...
Processing review 681...
Processing review 682...
Processing review 683...
Processing review 684...
Processing review 685...
Processing review 686...
Processing review 687...
Processing review 688...
Processing review 689...
Processing review 690...
Processing review 691...
Processing review 692...
Processing review 693...
Processing review 694...
Processing review 695...
Processing review 696...
Processing review 697...
Processing review 698...
Processing review 699...
Processing review 700...
Processing review 701...
Processing review 702...
Processing review 703...


In [39]:
value_counts = Counter(sentiments)

# Iterate through the Counter object and print in the desired format
for value, count in value_counts.items():
    print(f"{value} : {count}")

negative : 105
mixed : 74
positive : 664
neutral : 97
